<a href="https://colab.research.google.com/github/ziyi105/Car-Crash-Prediction-with-CRNN/blob/main/EE3201_Project_Group_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import the Videos and Text File to Local Google Collab**


# **Preprocessing**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# !mkdir -p /content/videos

!mkdir -p /content/video_frames

# !unzip /content/drive/MyDrive/CarCrash/videos/Crash-1500.zip -d /content/videos/car_crash
# !unzip /content/drive/MyDrive/CarCrash/videos/Normal.zip -d /content/videos/normal
!unzip /content/drive/MyDrive/full_dataset.zip -d /content/video_frames


Mounted at /content/drive
Archive:  /content/drive/MyDrive/full_dataset.zip
  inflating: /content/video_frames/normal_000940_frames.npz  
  inflating: /content/video_frames/normal_001065_frames.npz  
  inflating: /content/video_frames/crash_000139_frames.npz  
  inflating: /content/video_frames/normal_000656_frames.npz  
  inflating: /content/video_frames/normal_000268_frames.npz  
  inflating: /content/video_frames/crash_000049_frames.npz  
  inflating: /content/video_frames/normal_000446_frames.npz  
  inflating: /content/video_frames/normal_000249_frames.npz  
  inflating: /content/video_frames/normal_001449_frames.npz  
  inflating: /content/video_frames/normal_001468_frames.npz  
  inflating: /content/video_frames/normal_002368_frames.npz  
  inflating: /content/video_frames/normal_001018_frames.npz  
  inflating: /content/video_frames/normal_002804_frames.npz  
  inflating: /content/video_frames/normal_001588_frames.npz  
  inflating: /content/video_frames/normal_002667_frames.np

## Import Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
!pip install mlflow
import mlflow
from mlflow.models import infer_signature
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:
# export MLFLOW_TRACKING_URI=http://localhost:5000
mlflow.login()
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/ESP3201_car_crash_prediction")
# https://community.cloud.databricks.com
# ziyi_ng@u.nus.edu
# ESP3201project@

## Functions

In [ ]:
normal_video_path = "/content/videos/normal"

def extract_frames(video_path, first_frame=None, last_frame=None, output_size=(128, 128)):
  cap = cv2.VideoCapture(video_path)
  frames = []
  frame_count = 0

  # Detect border based on first frame so that border width is not recalculated for every frame
  # for some reason, if recalculated, the border width may change to 0 midway through the video
  ret, sample_frame = cap.read()
  height, width, _ = sample_frame.shape
  border_width = np.argmax(np.mean(sample_frame, axis=0) > 0)
  if border_width * 2 >= width:
      border_width = 0

  if not cap.isOpened():
      print(f"Error: Unable to open video file {video_path}")

  if first_frame is not None:
    cap.set(cv2.CAP_PROP_POS_FRAMES, first_frame)

  while cap.isOpened() and frame_count < 30:
      ret, frame = cap.read()
      if not ret:
          break

      frame = frame[:, border_width: int(width - border_width)]

      rgb_frame = frame[..., ::-1]

      resized_frame = cv2.resize(rgb_frame, output_size)
      normalized_frame = resized_frame / 255.0

      frames.append(normalized_frame)
      frame_count += 1
  cap.release()
  if frame_count == 30:
    frames = np.array(frames)
    return frames

  else:
    print(f"Warning: Only {len(frames)} frames were captured.")
    return None

def get_frames_before_accident(video_path, binlabels, frame_count=30):
    """
    Retrieve 30 frames before the first accident frame (where binlabels[i] == 1).
    """
    first_accident_frame = next((i for i, label in enumerate(binlabels) if label == 1), None)

    if first_accident_frame is None or first_accident_frame < frame_count:
        return None  # Not enough frames before accident or no accident frame

    first_frame = first_accident_frame - frame_count
    return first_frame, first_accident_frame

# Filter for egocentric videos according to text file data
def filter_egocentric_videos(txt_file):
  egocentric_txt = '/content/egocentric_videos.txt'
  egoinvolve_count = 0
  egocentric_videos = []
  with open(txt_file, 'r') as txt_file, open(egocentric_txt, 'w') as egocentric_txt:
      for line in txt_file:
          data = line.strip().split(',', 1)

          vidname = data[0].strip()

          temp = data[1].strip().split('[,', 1)
          data = temp[0].strip().split(']', 1)
          binlabels_str = data[0].replace('[','').replace(']','')
          data = data[1].replace(',', '', 1).split(',', 4)

          binlabels = list(map(int, binlabels_str.split(',')))

          startframe = data[0]
          youtubeID = data[1].strip()
          timing = data[2].strip()
          weather = data[3].strip()
          egoinvolve = data[4].strip()

          if egoinvolve == 'Yes':
              egoinvolve_count += 1
              egocentric_txt.write(line)
              egocentric_videos.append({
                  "vidname": vidname,
                  "binlabels": binlabels,
                  "startframe": startframe,
                  "youtubeID": youtubeID,
                  "timing": timing,
                  "weather": weather
              })

  return egocentric_videos

In [ ]:
# @title data quality check

import os
import pandas as pd

normal_video_dir = '/content/videos/normal'
crash_video_dir = '/content/videos/car_crash'

def calculate_sharpness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return laplacian_var

def remove_black_borders(frame):
    height, width, _ = frame.shape

    border_width = np.argmax(np.mean(frame, axis=0) > 0)

    if border_width * 2 >= width:
        border_width = 0  # Reset if the border width is too large

    frame = frame[:, border_width:int(width - border_width)]
    return frame

def check_video_quality(video_path):
    results = {}
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        results['video_name'] = os.path.basename(video_path)
        results['error'] = f"Error opening video file: {video_path}"
        return results  # Return error in results

    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    original_aspect_ratio = original_width / original_height
    print(f"Original dimensions: {original_width}x{original_height}")

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ret, frame = cap.read()

    if ret:
        frame = remove_black_borders(frame)

        plt.figure(figsize=(6, 6))
        plt.title(f"Processed Frame from {os.path.basename(video_path)}")
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

        resized_frame = cv2.resize(frame, (128, 128))
        resized_aspect_ratio = 128 / 128

        print(f"Original aspect ratio: {original_aspect_ratio:.2f}, Resized aspect ratio: {resized_aspect_ratio:.2f}")

        plt.figure(figsize=(6, 6))
        plt.title(f"Resized Frame (128x128) from {os.path.basename(video_path)}")
        plt.imshow(cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

        original_sharpness = calculate_sharpness(frame)
        resized_sharpness = calculate_sharpness(resized_frame)
        print(f"Original Sharpness (Laplacian variance): {original_sharpness:.2f}")
        print(f"Resized Sharpness (Laplacian variance): {resized_sharpness:.2f}")

        results['video_name'] = os.path.basename(video_path)
        results['original_sharpness'] = original_sharpness
        results['resized_sharpness'] = resized_sharpness
        results['error'] = None
    else:
        results['video_name'] = os.path.basename(video_path)
        results['error'] = f"Error reading frame from {video_path}"

    cap.release()
    return results

egocentric_videos = filter_egocentric_videos("/content/Crash-1500.txt")
pos_sample = [34, 984, 200, 495, 881, 440, 516, 688, 1151, 1422]
neg_sample = [314, 2158, 1281, 1638, 147, 2123, 1773, 492, 1254, 1879]

results_summary = []

for i in neg_sample:
    video_name = f"{i:06d}.mp4"
    video_path = os.path.join(normal_video_dir, video_name)
    print(f"\nChecking video: {video_name}")
    result = check_video_quality(video_path)
    results_summary.append(result)

for i in pos_sample:
    video_name = f"{i:06d}.mp4"
    video_path = os.path.join(crash_video_dir, video_name)
    print(f"\nChecking random egocentric video: {video_name}")
    result = check_video_quality(video_path)
    results_summary.append(result)

results_df = pd.DataFrame(results_summary)

print("\nSummary of Video Quality Checks:")
print(results_df)


## **Processing**

In [ ]:
import random
import numpy as np

total_videos = 3000
selected_videos = random.sample(range(total_videos), 250)
count = 0
for i in range(1, total_videos+1):
  video_id = f"{i:06d}"
  video_path = f"/content/videos/normal/{video_id}.mp4"
  print(video_path)
  frames_file_path = f"/content/video_frames/normal_{video_id}_frames.npz"
  np.savez_compressed(frames_file_path,
                      frames=extract_frames(video_path=video_path),
                      label = 0)
  count += 1
  if count % 10 == 0 and count > 0:
    print(f"Processed {count} normal videos.")

/content/videos/normal/000001.mp4
/content/videos/normal/000002.mp4
/content/videos/normal/000003.mp4
/content/videos/normal/000004.mp4
/content/videos/normal/000005.mp4
/content/videos/normal/000006.mp4
/content/videos/normal/000007.mp4
/content/videos/normal/000008.mp4
/content/videos/normal/000009.mp4
/content/videos/normal/000010.mp4
Processed 10 normal videos.
/content/videos/normal/000011.mp4
/content/videos/normal/000012.mp4
/content/videos/normal/000013.mp4
/content/videos/normal/000014.mp4
/content/videos/normal/000015.mp4
/content/videos/normal/000016.mp4
/content/videos/normal/000017.mp4
/content/videos/normal/000018.mp4
/content/videos/normal/000019.mp4
/content/videos/normal/000020.mp4
Processed 20 normal videos.
/content/videos/normal/000021.mp4
/content/videos/normal/000022.mp4
/content/videos/normal/000023.mp4
/content/videos/normal/000024.mp4
/content/videos/normal/000025.mp4
/content/videos/normal/000026.mp4
/content/videos/normal/000027.mp4
/content/videos/normal/000

In [ ]:
egocentric_videos = filter_egocentric_videos("/content/Crash-1500.txt")
selected_videos = random.sample(egocentric_videos, 250)

for i, video in enumerate(egocentric_videos):
    video_name = video["vidname"]
    video_path = f"/content/videos/car_crash/{video_name}.mp4"
    start_frame, end_frame = get_frames_before_accident(video_path, video["binlabels"])
    frames_file_path = f"/content/video_frames/crash_{video_name}_frames.npz"
    np.savez_compressed(frames_file_path,
                        frames=extract_frames(video_path, first_frame=start_frame, last_frame=end_frame),
                        label=1)
    if (i+1) % 10 == 0 and i > 0:
        print(f"Processed {i+1} car crash videos.")

In [ ]:
# @title Default title text
# Process ego-invovlve car crash videos
egocentric_videos = filter_egocentric_videos("/content/Crash-1500.txt")
#for i in range(len(egocentric_videos)):
for i in range(101, 151):
  video = egocentric_videos[i]
  video_name = video["vidname"]
  video_path = f"/content/videos/car_crash/{video_name}.mp4";
  start_frame, end_frame = get_frames_before_accident(video_path, video["binlabels"])
  frames_file_path = f"/content/video_frames/crash_{video_name}_frames.npz"
  np.savez_compressed(frames_file_path,
                      frames=extract_frames(video_path, first_frame=start_frame, last_frame=end_frame),
                      label = 1)
  if (i+1) % 10 == 0 and i > 0:
    print(f"Processed {i+1} car crash videos.")



In [ ]:
import shutil
shutil.make_archive('video_frames', 'zip', '/content/video_frames')
# from google.colab import files
# files.download('video_frames.zip')

In [ ]:
# @title Visualizing frames
egocentric_videos = filter_egocentric_videos("/content/Crash-1500.txt")
video = egocentric_videos[0]
video_name = video["vidname"]
video_path = f"/content/videos/car_crash/{video_name}.mp4";
start_frame, end_frame = get_frames_before_accident(video_path, video["binlabels"])

frames = []
frame_count = 0
cap = cv2.VideoCapture(video_path)
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
while frame_count < 30:
  ret, frame = cap.read()
  frame = frame[..., ::-1]
  frames.append(frame)
  frame_count += 1
cap.release()

plt.figure(figsize=(10, 10))
for i in range(min(30, len(frames))):
    plt.subplot(5, 6, i + 1)
    plt.imshow(frames[i])
    plt.axis('off')
plt.show()

frames = np.load('/content/video_frames/crash_000001_frames.npz')['frames']
plt.figure(figsize=(10, 10))
for i in range(min(30, len(frames))):
    plt.subplot(5, 6, i + 1)
    plt.imshow(frames[i])
    plt.axis('off')
plt.show()

# **Train Test Spilt Data**

In [ ]:
!unzip /content/video_frames.zip -d /content/video_frames

unzip:  cannot find or open /content/video_frames.zip, /content/video_frames.zip.zip or /content/video_frames.zip.ZIP.


In [ ]:
import gc
gc.collect()

%reset -f

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
from sklearn.model_selection import train_test_split
import zipfile
import os

class CarCrashDataset(Dataset):
    def __init__(self, file_paths):
        """
        Args:
            file_paths (list of str): Each path is the file path to a .npz file with frames and label.
        """
        self.file_paths = file_paths

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        data = np.load(self.file_paths[idx])
        frames = torch.tensor(data['frames'], dtype=torch.float32).permute(0, 3, 1, 2)  # Shape (30, C, H, W)
        label = torch.tensor(data['label'], dtype=torch.float32)
        return frames, label

file_paths = glob.glob('/content/video_frames/*.npz')
print(f"Number of .npz files found: {len(file_paths)}")

positive_files = [f for f in file_paths if np.load(f)['label'] == 1]
negative_files = [f for f in file_paths if np.load(f)['label'] == 0]

train_pos, temp_pos = train_test_split(positive_files, test_size=0.3, random_state=42)
val_pos, eval_pos = train_test_split(temp_pos, test_size=0.5, random_state=42)

train_neg, temp_neg = train_test_split(negative_files, test_size=0.3, random_state=42)
val_neg, eval_neg = train_test_split(temp_neg, test_size=0.5, random_state=42)

X_train = train_pos + train_neg
X_val = val_pos + val_neg
X_eval = eval_pos + eval_neg

np.random.shuffle(X_train)
np.random.shuffle(X_val)
np.random.shuffle(X_eval)

train_dataset = CarCrashDataset(X_train)
val_dataset = CarCrashDataset(X_val)
eval_dataset = CarCrashDataset(X_eval)

print(f"Train dataset length: {len(train_dataset)}")
print(f"Validation dataset length: {len(val_dataset)}")
print(f"Evaluation dataset length: {len(eval_dataset)}")

def count_labels(dataset):
    positive_count = 0
    negative_count = 0
    for _, label in dataset:
        if label.item() == 1:
            positive_count += 1
        else:
            negative_count += 1
    return positive_count, negative_count

train_positive, train_negative = count_labels(train_dataset)
val_positive, val_negative = count_labels(val_dataset)
eval_positive, eval_negative = count_labels(eval_dataset)
print(f"Train set: Positive = {train_positive}, Negative = {train_negative}")
print(f"Validation set: Positive = {val_positive}, Negative = {val_negative}")
print(f"Evaluation set: Positive = {eval_positive}, Negative = {eval_negative}")

def data_loader(params):
    train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True, drop_last=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=params["batch_size"], shuffle=False, drop_last=False, num_workers=0)
    eval_loader = DataLoader(eval_dataset, batch_size=params["batch_size"], shuffle=False, drop_last=True, num_workers=0)

    return train_loader, val_loader, eval_loader

def save_dataset_as_zip(dataset, dataset_name, destination_folder='/content/zipped_datasets'):
    """
    Zips the files in the dataset and saves them as a single zip file.

    Args:
        dataset (Dataset): Dataset object containing file paths to be zipped.
        dataset_name (str): Name of the zip file to be created (e.g., "train", "val", "eval").
        destination_folder (str): Folder where the zip files will be stored.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    zip_path = os.path.join(destination_folder, f"{dataset_name}.zip")

    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for file_path, _ in dataset:
            file_name = os.path.basename(file_path)
            zipf.write(file_path, arcname=file_name)

    print(f"{dataset_name}.zip created at {zip_path}")

# # Saving each dataset as a zip file
# save_dataset_as_zip(train_dataset, "train_dataset")
# save_dataset_as_zip(val_dataset, "val_dataset")
# save_dataset_as_zip(eval_dataset, "eval_dataset")


Number of .npz files found: 3801
Train dataset length: 2660
Validation dataset length: 570
Evaluation dataset length: 571
Train set: Positive = 560, Negative = 2100
Validation set: Positive = 120, Negative = 450
Evaluation set: Positive = 121, Negative = 450


# **Model**

Import Model Libraries

Model Definition

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np
import gc

class CRNN(nn.Module):
    def __init__(self, input_shape):
        super(CRNN, self).__init__()

        self.dropout_lstm = nn.Dropout(p=0.2)
        self.dropout_fc = nn.Dropout(p=0.3)

        self.mobilenetv2 = models.mobilenet_v2(pretrained=True).features

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.lstm1 = nn.LSTM(input_size=1280, hidden_size=128, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128, hidden_size=64, batch_first=True)

        self.fc = nn.Linear(64, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        batch_size, sequence_length, channels, height, width = x.size()

        cnn_features = []

        for t in range(sequence_length):
            frame_features = self.mobilenetv2(x[:, t, :, :, :])
            frame_features = self.global_avg_pool(frame_features)
            frame_features = frame_features.view(batch_size, -1)
            cnn_features.append(frame_features)

        cnn_features = torch.stack(cnn_features, dim=1)

        lstm_out, _ = self.lstm1(cnn_features)
        lstm_out = self.dropout_lstm(lstm_out)

        lstm_out, _ = self.lstm2(lstm_out)
        lstm_out = self.dropout_lstm(lstm_out)

        output = self.fc(lstm_out[:, -1, :])
        output = self.dropout_fc(output)

        output = self.sigmoid(output)

        return output

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from sklearn.metrics import confusion_matrix
from torch.optim.lr_scheduler import ReduceLROnPlateau

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print("Early stopping triggered.")
                self.early_stop = True


def train_model(model, params, train_loader, val_loader):
    epochs = params['epochs']
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])
    pos_weight = torch.tensor([4.5]).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    # criterion = nn.BCELoss()
    early_stopping = EarlyStopping(patience=5, min_delta=0.001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5)

    best_val_loss = float('inf')
    best_weights = None
    accumulation_steps = params.get('accumulation_steps', 1)

    for epoch in range(epochs):
        # Training step
        model.train()
        running_loss = 0.0

        for i, (frames, labels) in enumerate(train_loader):
            frames, labels = frames.to(device), labels.to(device)
            if len(labels.shape) == 1:
                labels = labels.view(-1, 1)

            optimizer.zero_grad()
            outputs = model(frames)
            loss = criterion(outputs, labels)
            loss = loss / accumulation_steps
            loss.backward()

            running_loss += loss.item()
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

        # Validation step
        model.eval()
        val_loss = 0.0
        all_labels = []
        all_preds = []

        with torch.no_grad():
            for frames, labels in val_loader:
                frames, labels = frames.to(device), labels.to(device)
                outputs = torch.sigmoid(model(frames))
                val_loss += criterion(outputs.squeeze(), labels.view(-1)).item()

                preds = (outputs > 0.5).float()
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        val_loss /= len(val_loader)
        scheduler.step(val_loss)
        print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")

        tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
        false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
        false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0

        print(f"False Positives: {fp}, False Negatives: {fn}")
        print(f"False Positive Rate: {false_positive_rate:.4f}, False Negative Rate: {false_negative_rate:.4f}")

        mlflow.log_metric("validation_loss", val_loss, step=epoch)
        # mlflow.log_metric("false_positives", fp, step=epoch)
        # mlflow.log_metric("false_negatives", fn, step=epoch)
        # mlflow.log_metric("false_positive_rate", false_positive_rate, step=epoch)
        # mlflow.log_metric("false_negative_rate", false_negative_rate, step=epoch)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_weights = model.state_dict()

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Stopping training early...")
            break

    torch.save(best_weights, "best_model_weights.pth")
    mlflow.log_artifact("best_model_weights.pth")
    os.remove("best_model_weights.pth")


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

def evaluate_model(model, eval_loader, params):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0
    total_samples = 0

    all_labels = []
    all_predictions = []

    criterion = nn.BCELoss()

    with torch.no_grad():
        for frames, labels in eval_loader:
            frames, labels = frames.to(device), labels.to(device)

            outputs = model(frames)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted.squeeze() == labels).sum().item()

            outputs = outputs.view(-1)
            loss = criterion(outputs, labels.view(-1))
            total_loss += loss.item() * labels.size(0)
            total_samples += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    avg_loss = total_loss / total_samples
    accuracy = correct / total
    precision = precision_score(all_labels, all_predictions, zero_division=0)
    recall = recall_score(all_labels, all_predictions, zero_division=0)
    f1 = f1_score(all_labels, all_predictions, zero_division=0)

    tn, fp, fn, tp = confusion_matrix(all_labels, all_predictions).ravel()

    true_positives_count = np.sum(np.array(all_labels) == 1)
    true_negatives_count = np.sum(np.array(all_labels) == 0)
    predicted_positives_count = np.sum(np.array(all_predictions) == 1)
    predicted_negatives_count = np.sum(np.array(all_predictions) == 0)

    # print(f'Average Loss: {avg_loss:.4f}')
    # print(f'Accuracy: {accuracy:.4f}')
    # print(f'Confusion Matrix:')
    # print(f'True Negatives: {tn}, False Positives: {fp}')
    # print(f'False Negatives: {fn}, True Positives: {tp}')
    # print(f'Total Positive Labels: {true_positives_count}, Total Negative Labels: {true_negatives_count}')
    # print(f'Total Positive Predictions: {predicted_positives_count}, Total Negative Predictions: {predicted_negatives_count}')

    return avg_loss, accuracy, precision, recall, f1, tn, fp, fn, tp


In [ ]:
def train_eval_model(params, model):
    # with mlflow.start_run(nested=False):
        model.to(device)
        train_loader, val_loader, eval_loader = data_loader(params)

        for param_name, param_value in params.items():
            mlflow.log_param(param_name, param_value)

        train_model(model, params, train_loader, val_loader)
        torch.cuda.empty_cache()
        gc.collect()
        avg_loss, accuracy, precision, recall, f1, tn, fp, fn, tp = evaluate_model(model, eval_loader, params)
        torch.cuda.empty_cache()
        gc.collect()

        mlflow.log_metric("average_loss", avg_loss)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("tn", tn)
        mlflow.log_metric("fp", fp)
        mlflow.log_metric("fn", fn)
        mlflow.log_metric("tp", tp)
        return {'loss': avg_loss, 'status': STATUS_OK, 'model': model}

def objective(params):
    torch.cuda.empty_cache()
    gc.collect()
    if mlflow.active_run() is not None:
        print("Ending previous active run")
        mlflow.end_run()

    try:
        print("Starting new MLflow run")
        with mlflow.start_run():
            mlflow.set_tag("mlflow.runName", f"(weight=4.5, dropout=0.3, epoch=30) Trial with params: {params}")

            for param_name, param_value in params.items():
                mlflow.log_param(param_name, param_value)

            results = train_eval_model(params, CRNN(input_shape=(30, 128, 128, 3)).to(device))
            torch.cuda.empty_cache()
            gc.collect()

    finally:
        print("Ensuring run ends")
        mlflow.end_run()
        torch.cuda.empty_cache()
        gc.collect()

    return results

In [ ]:
# Hyperparameters sweeping
import os
import pickle
from hyperopt import fmin, tpe, Trials
from hyperopt import STATUS_OK
from google.colab import files
import numpy as np
from hyperopt import hp
import mlflow

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
space = {
    "epochs": hp.choice("epochs", [30]),
    "batch_size": hp.choice("batch_size", [4]),
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-4)),
    "accumulation_steps": 4,
}

trials_file = 'trials.pkl'

if os.path.exists(trials_file):
    with open(trials_file, 'rb') as f:
        trials = pickle.load(f)
else:
    trials = Trials()

max_evals = 2
mlflow.set_experiment("/datasize-500")

with mlflow.start_run():
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals + len(trials.trials),
        trials=trials
    )

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    mlflow.log_params(best)
    # mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)
    # mlflow.pytorch.log_model(best_run["model"], "model")

    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

    with open(trials_file, 'wb') as f:
      pickle.dump(trials, f)

    files.download(trials_file)

best_trial = trials.best_trial
print("Best Trial:", best_trial)


Ending previous active run
  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

2024/11/13 22:12:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run dashing-boar-32 at: https://community.cloud.databricks.com/ml/experiments/3052994282681228/runs/adc50efc7e82486abc8662ce175c5f1c.

2024/11/13 22:12:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/3052994282681228.



Starting new MLflow run
  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth

  0%|          | 0.00/13.6M [00:00<?, ?B/s]

 56%|#####6    | 7.62M/13.6M [00:00<00:00, 79.8MB/s]

100%|##########| 13.6M/13.6M [00:00<00:00, 95.3MB/s]
2024/11/13 22:12:23 INFO mlflow.tracking._tracking_service.client: 🏃 View

Ensuring run ends
  0%|          | 0/2 [00:05<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [ ]:
# Train model with predefined params
import os
import pickle
from hyperopt import fmin, tpe, Trials
from hyperopt import STATUS_OK
from google.colab import files
import numpy as np
from hyperopt import hp
import mlflow
import gc

mlflow.set_experiment("/datasize-500")
params = {
    "epochs": 40,
    "batch_size": 16,
    "lr": 9.007553054088038e-05,
    "accumulation_steps": 4,
}

with mlflow.start_run():
  mlflow.set_tag("mlflow.runName", f"unbalanced data (with scheduler) without weighted loss Trial with params: {params}")
  for param_name, param_value in params.items():
    mlflow.log_param(param_name, param_value)
  train_eval_model(params, CRNN(input_shape=(30, 128, 128, 3)))

In [ ]:
mlflow.end_run()